### build_embeddings.py

import os
import numpy as np
from utils_facenet import embed_from_path

DATA_DIR = "data/train"

embeddings = []
labels = []

for person in os.listdir(DATA_DIR):
    person_dir = os.path.join(DATA_DIR, person)
    if not os.path.isdir(person_dir):
        continue

    print(f"Proses folder: {person}")

    for img_name in os.listdir(person_dir):
        img_path = os.path.join(person_dir, img_name)
        print("  →", img_path)

        emb = embed_from_path(img_path)
        if emb is None:
            print("  [X] Wajah tidak terdeteksi. Dilewati.")
            continue

        embeddings.append(emb)
        labels.append(person)

embeddings = np.array(embeddings)
labels = np.array(labels)

print("\nJumlah embedding:", len(embeddings))

np.savez("embeddings.npz", embeddings=embeddings, labels=labels)
print("embeddings.npz berhasil disimpan!")

#### Analisis

Script build_embeddings.py berfungsi untuk memproses dataset wajah di folder data/train dan menghasilkan file embeddings.npz yang berisi vektor embedding serta label masing-masing gambar. Program memulai dengan membaca setiap folder yang merepresentasikan identitas seseorang, lalu masuk ke setiap file gambar di dalamnya. Untuk setiap gambar, script memanggil fungsi embed_from_path() yang berasal dari utils_facenet, yang bertugas mendeteksi wajah dan menghasilkan embedding numerik (biasanya panjang 128 atau 512 tergantung model). Jika wajah tidak terdeteksi, gambar dilewati agar hasil dataset tetap bersih. Embedding yang valid dimasukkan ke list embeddings, dan nama folder (identitas orang) dimasukkan ke list labels. Setelah semua folder selesai diproses, kedua list tersebut diubah menjadi array NumPy dan disimpan ke satu file embeddings.npz, sehingga memudahkan dipakai pada model lain seperti KNN, SVM, atau sistem verifikasi wajah. Struktur ini memastikan dataset embedding konsisten, mudah di-load, dan efisien digunakan pada tahap training maupun prediksi berikutnya.

### convert_npz_to_npy.py

import numpy as np

data = np.load("embeddings.npz")

X = data["embeddings"]
y = data["labels"]

np.save("X_train.npy", X)
np.save("y_train.npy", y)

print("Berhasil membuat X_train.npy dan y_train.npy")
print("Shape X:", X.shape)
print("Jumlah label:", len(y))

#### Analisis

Script convert_npz_to_npy.py berfungsi untuk mengambil data embedding wajah yang sebelumnya disimpan dalam satu file embeddings.npz dan memecahnya menjadi dua file terpisah, yaitu X_train.npy dan y_train.npy. Saat file embeddings.npz dibuka menggunakan np.load, script mengambil dua array utama: embeddings sebagai fitur (X) dan labels sebagai identitas atau target (y). Setelah itu, masing-masing array disimpan kembali ke format .npy, yang lebih sederhana dan cepat diakses ketika digunakan pada model machine learning seperti KNN atau SVM. File X_train.npy akan berisi seluruh vektor embedding numerik, sementara y_train.npy menyimpan label orang untuk setiap embedding. Terakhir, script menampilkan ukuran X dan jumlah label sebagai verifikasi bahwa data berhasil diekstrak dengan benar. Proses ini memastikan dataset lebih modular, lebih mudah di-load secara terpisah, dan siap dipakai untuk tahap training model pengenalan wajah.

### evaluate.py

import os
import numpy as np
import joblib
from utils_facenet import embed_from_path

VAL_DIR = "data/val"
clf = joblib.load("svm_model.pkl")

correct = 0
total = 0

for person in os.listdir(VAL_DIR):
    person_dir = os.path.join(VAL_DIR, person)
    if not os.path.isdir(person_dir):
        continue

    print(f"Testing folder: {person}")

    for img_name in os.listdir(person_dir):
        img_path = os.path.join(person_dir, img_name)

        emb = embed_from_path(img_path)
        if emb is None:
            print(f"  [X] Gagal deteksi wajah: {img_name}")
            continue

        pred = clf.predict([emb])[0]
        print(f"  → {img_name} | Prediksi: {pred} | Label benar: {person}")

        if pred == person:
            correct += 1
        total += 1

print("\nTotal data:", total)
print("Benar:", correct)
print("Akurasi:", correct / total if total > 0 else 0)

#### Analisis

Script evaluate.py digunakan untuk menghitung akurasi model pengenalan wajah berbasis SVM menggunakan dataset validasi yang berada di folder data/val. Program memulai dengan memuat model SVM yang telah dilatih sebelumnya melalui file svm_model.pkl. Kemudian, untuk setiap folder di dalam data/val—yang masing-masing mewakili identitas seseorang—script membaca seluruh gambar di dalamnya. Setiap gambar diproses menggunakan fungsi embed_from_path() untuk menghasilkan embedding wajah; jika wajah gagal terdeteksi, gambar dilewati agar tidak memengaruhi hasil evaluasi. Setelah embedding berhasil dibuat, model SVM melakukan prediksi identitas dan hasilnya dibandingkan dengan nama folder sebagai label ground truth. Program juga mencetak hasil prediksi setiap gambar sehingga memudahkan debugging. Setiap prediksi yang benar akan menambah hitungan correct, sedangkan total gambar valid akan menambah total. Di akhir proses, script menampilkan total data yang diuji, jumlah prediksi benar, dan nilai akurasi keseluruhan. Dengan cara ini, pengguna dapat menilai seberapa baik model SVM mengenali wajah pada data baru yang tidak digunakan saat training.

### predict_knn.py

import numpy as np
import joblib
from utils_facenet import embed_from_path
import sys

MODEL_PATH = "knn_model.pkl"

#Load model KNN
knn = joblib.load(MODEL_PATH)

def predict_image(img_path):
    emb = embed_from_path(img_path)
    if emb is None:
        print("❌ Wajah tidak terdeteksi.")
        return
    
    emb = emb.reshape(1, -1)  # bentuk menjadi (1, 512)
    pred = knn.predict(emb)[0]

    # hitung jarak untuk confidence
    dist, i# train_knn.py
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
import joblib

#======================
#Load dataset dari embeddings.npz
#======================
data = np.load("embeddings.npz")

X = data["embeddings"]   # ← sesuai key
y = data["labels"]       # ← sesuai key

print("Shape X:", X.shape)
print("Shape y:", y.shape)

#======================
#Training KNN
#======================
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X, y)

#======================
#Simpan model
#======================
joblib.dump(knn, "knn_model.pkl")
print("Model KNN berhasil disimpan! → knn_model.pkl")dx = knn.kneighbors(emb, n_neighbors=1, return_distance=True)
    conf = 1 / (1 + dist[0][0])

    print("\nMemprediksi gambar:", img_path)
    print("Prediksi:", pred)
    print("Confidence:", round(conf, 3))

if __name__ == "__main__":
    if len(sys.argv) < 2:
        print("Usage: python predict_knn.py <path_gambar>")
    else:
        predict_image(sys.argv[1])

#### Analisis

Script ini dirancang untuk melakukan prediksi identitas wajah menggunakan model KNN yang telah dilatih sebelumnya dan disimpan dalam file knn_model.pkl. Pertama, program memuat model KNN menggunakan joblib.load, kemudian mendefinisikan fungsi predict_image() yang menerima path gambar sebagai input. Di dalam fungsi tersebut, gambar diubah menjadi embedding wajah menggunakan embed_from_path(). Jika wajah tidak terdeteksi, proses dihentikan agar prediksi tidak salah. Embedding kemudian di-reshape menjadi bentuk dua dimensi (1, 512) karena model KNN hanya menerima input dalam bentuk batch, bukan satu dimensi tunggal. Setelah itu, model melakukan prediksi label identitas menggunakan knn.predict(). Untuk memberikan informasi tingkat keyakinan (confidence), script menghitung jarak terdekat ke tetangga terdekat KNN dengan knn.kneighbors(). Nilai confidence dihitung menggunakan rumus sederhana 1 / (1 + distance), di mana semakin kecil jarak, semakin tinggi confidence. Program juga menampilkan prediksi, confidence, dan nama file gambar untuk memudahkan debugging. Pada bagian main, script menerima path gambar melalui argumen command-line dan menjalankan prediksi hanya jika argumen diberikan. Dengan struktur seperti ini, script sangat praktis digunakan untuk pengujian cepat terhadap satu gambar tanpa perlu membuat pipeline tambahan.

### train_classifier.py

import numpy as np
from sklearn.svm import SVC
import joblib

#Load embeddings
data = np.load("embeddings.npz")
X = data["embeddings"]   # embedding 512-dim
y = data["labels"]        # nama orangnya

print("Shape X:", X.shape)
print("Labels:", y)

#Create classifier
clf = SVC(kernel="linear", probability=True)

#Train
print("Training classifier...")
clf.fit(X, y)

#Save model
joblib.dump(clf, "svm_model.pkl")
print("Model SVM berhasil disimpan sebagai svm_model.pkl!")

#### Analisis

Script train_classifier.py berfungsi untuk melatih model klasifikasi wajah menggunakan algoritma Support Vector Machine (SVM) berbasis kernel linear. Program dimulai dengan memuat file embeddings.npz, yang berisi dua array penting: embeddings (X) sebagai fitur wajah berdimensi tinggi dan labels (y) sebagai identitas orang dari setiap embedding. Setelah menampilkan ukuran data untuk verifikasi, script membuat model SVM dengan parameter kernel="linear" yang sangat cocok untuk data embedding wajah karena sifat ruang vektor yang sudah relatif terpisah secara linear. Opsi probability=True ditambahkan agar model mampu memberikan output probabilitas prediksi saat digunakan nanti. Proses training kemudian dijalankan menggunakan clf.fit(X, y), di mana model mempelajari hubungan antara embedding wajah dan identitas aslinya. Setelah pelatihan selesai, model disimpan ke dalam file svm_model.pkl melalui joblib.dump, sehingga dapat digunakan kembali di script lain seperti evaluasi atau prediksi real-time tanpa harus melatih ulang. Dengan struktur yang sederhana dan efisien, script ini menyediakan proses training yang jelas, mudah dipahami, dan langsung siap digunakan dalam sistem face recognition.

### train_knn.py

import numpy as np
from sklearn.neighbors import KNeighborsClassifier
import joblib

#======================
#Load dataset dari embeddings.npz
#======================
data = np.load("embeddings.npz")

X = data["embeddings"]   # ← sesuai key
y = data["labels"]       # ← sesuai key

print("Shape X:", X.shape)
print("Shape y:", y.shape)

#======================
#Training KNN
#======================
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X, y)

#======================
#Simpan model
#======================
joblib.dump(knn, "knn_model.pkl")
print("Model KNN berhasil disimpan! → knn_model.pkl")

#### Analisis

Script train_knn.py digunakan untuk melatih model klasifikasi wajah berbasis K-Nearest Neighbors (KNN) menggunakan embedding wajah yang tersimpan dalam file embeddings.npz. Program diawali dengan memuat dataset embedding dan label, yang masing-masing mewakili vektor fitur wajah berdimensi tinggi serta identitas orangnya. Informasi shape ditampilkan untuk memastikan data telah terbaca dengan benar. Kemudian, script membuat model KNN dengan parameter n_neighbors=1, yang berarti setiap prediksi akan bergantung pada jarak embedding terdekat tunggal dalam ruang vektor. Penggunaan K=1 sangat umum dalam face recognition karena embedding FaceNet cenderung terdistribusi dengan baik sehingga jarak terdekat biasanya cukup untuk memisahkan identitas. Model KNN lalu dilatih menggunakan .fit(X, y), di mana proses ini sebenarnya hanya menyimpan data tanpa membangun hyperplane, karena KNN merupakan lazy learner. Setelah training selesai, model disimpan sebagai knn_model.pkl menggunakan joblib agar dapat dipakai di script lain, seperti predict_knn.py. Dengan struktur sederhana namun efektif, script ini memungkinkan pembuatan model face recognition berbasis jarak yang cepat, mudah, dan sangat cocok untuk sistem real-time atau skala kecil–menengah.

### utils_facenet.py

import torch, numpy as np, cv2
from PIL import Image
from facenet_pytorch import MTCNN, InceptionResnetV1

device = 'cuda' if torch.cuda.is_available() else 'cpu'

#Detector & aligner
mtcnn = MTCNN(image_size=160, margin=20, post_process=True, device=device)

#Embedder (512-dim)
embedder = InceptionResnetV1(pretrained='vggface2').eval().to(device)

def read_img_bgr(path):
    img = cv2.imread(path)  # BGR
    if img is None:
        raise ValueError(f"Gagal baca: {path}")
    return img

def bgr_to_pil(img_bgr):
    return Image.fromarray(cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB))

@torch.no_grad()
def face_align(img_bgr):
    """Return aligned face as PIL.Image (160x160) or None if not found."""
    pil = bgr_to_pil(img_bgr)
    aligned = mtcnn(pil)  # tensor [3,160,160] atau None
    return aligned

@torch.no_grad()
def embed_face_tensor(face_tensor):
    if face_tensor is None:
        return None
    face_tensor = face_tensor.unsqueeze(0).to(device)
    emb = embedder(face_tensor)
    return emb.squeeze(0).cpu().numpy()

@torch.no_grad()
def embed_from_path(path):
    img = read_img_bgr(path)
    face = face_align(img)
    if face is None:
        return None
    return embed_face_tensor(face)

def cosine_similarity(a, b, eps=1e-8):
    a = a / (np.linalg.norm(a) + eps)
    b = b / (np.linalg.norm(b) + eps)
    return float(np.dot(a, b))

#### Analisis

File utils_facenet.py berisi seluruh utilitas inti yang digunakan untuk membaca gambar, mendeteksi wajah, melakukan alignment, dan menghasilkan embedding wajah menggunakan model FaceNet berbasis InceptionResnetV1. Script dimulai dengan memilih device (“cuda” jika GPU tersedia, jika tidak menggunakan CPU) sehingga sistem dapat bekerja optimal di berbagai lingkungan. Untuk proses deteksi dan alignment, script menggunakan MTCNN, yang bertugas mencari wajah dalam gambar, memotong area wajah, dan menormalkannya ke ukuran 160×160. Selanjutnya, model InceptionResnetV1 dengan pretrained vggface2 dipakai sebagai embedder yang menghasilkan vektor embedding berdimensi 512, yang menjadi representasi numerik unik untuk setiap wajah.

Fungsi read_img_bgr() menggunakan OpenCV untuk membaca gambar dalam format BGR, sedangkan bgr_to_pil() mengkonversinya menjadi PIL RGB agar bisa diproses MTCNN. Fungsi face_align() menjalankan deteksi dan alignment wajah; jika wajah tidak ditemukan, fungsi mengembalikan None, yang mencegah error saat script lain memanggil embedding. Fungsi embed_face_tensor() menerima tensor wajah ter-align, menjalankannya melalui model embedder, lalu mengembalikan array numpy 512 dimensi. Fungsi utama embed_from_path() menggabungkan semua langkah: membaca gambar, mendeteksi wajah, melakukan alignment, hingga menghasilkan embedding akhir. Script juga menyediakan fungsi cosine_similarity() untuk menghitung kesamaan antara dua embedding, metode umum dalam face verification. Secara keseluruhan, file ini adalah inti dari seluruh pipeline pengenalan wajah karena menyediakan deteksi, preprocessing, dan ekstraksi fitur yang konsisten untuk digunakan pada KNN, SVM, maupun verifikasi berbasis jarak.

### verify_cli.py

import argparse
from utils_facenet import embed_from_path, cosine_similarity

parser = argparse.ArgumentParser()
parser.add_argument("img1")
parser.add_argument("img2")
parser.add_argument("--th", type=float, default=0.85)
args = parser.parse_args()

e1 = embed_from_path(args.img1)
e2 = embed_from_path(args.img2)

if e1 is None or e2 is None:
    print("❌ Wajah tidak terdeteksi.")
else:
    sim = cosine_similarity(e1, e2)
    print(f"Similarity = {sim:.4f}")
    print("MATCH" if sim >= args.th else "NO MATCH")

#### Analisis

Script verify_cli.py berfungsi sebagai alat verifikasi wajah berbasis command-line yang membandingkan dua gambar dan menentukan apakah keduanya merupakan wajah orang yang sama. Program menggunakan argparse untuk menerima dua path gambar serta threshold kesamaan opsional (--th) dengan nilai default 0.85. Dua embedding wajah dihasilkan melalui fungsi embed_from_path() dari utils_facenet.py. Jika salah satu gambar tidak mengandung wajah yang berhasil dideteksi oleh MTCNN, script langsung menampilkan pesan error agar proses verifikasi tidak menghasilkan keputusan keliru.
Ketika kedua wajah berhasil diekstraksi, script menghitung skor kemiripan menggunakan cosine similarity, sebuah metrik yang umum digunakan dalam sistem FaceNet karena dapat mengukur jarak angular antara dua vektor embedding. Hasil similarity dicetak dengan presisi empat desimal untuk transparansi. Setelah itu, script membandingkan nilai similarity tersebut dengan threshold. Jika similarity lebih besar atau sama dengan threshold, wajah dianggap MATCH (orang yang sama); jika lebih rendah, dianggap NO MATCH. Dengan desain yang sederhana namun efektif, script ini sangat cocok untuk membangun fitur verifikasi wajah mandiri atau proses uji cepat antara dua gambar tanpa melibatkan model klasifikasi seperti SVM atau KNN.

### verify_pair.py

from utils_facenet import embed_from_path, cosine_similarity

img1 =  "samples/iin_test.jpeg"    # ganti sesuai nama file kamu
img2 = "samples/ami_test.jpeg"   # ganti sesuai nama file kamu

emb1 = embed_from_path(img1)
emb2 = embed_from_path(img2)

if emb1 is None or emb2 is None:
    print("Wajah tidak terdeteksi pada salah satu gambar.")
else:
    sim = cosine_similarity(emb1, emb2)
    print("Cosine similarity:", sim)

    threshold = 0.85
    print("Match?", "YA" if sim >= threshold else "TIDAK")

#### Analisis

Script verify_pair.py merupakan alat sederhana untuk melakukan verifikasi wajah dengan membandingkan dua gambar secara langsung tanpa menggunakan command-line argument seperti pada verify_cli.py. Program ini secara eksplisit mendefinisikan dua path gambar (img1 dan img2), kemudian menghasilkan embedding masing-masing gambar menggunakan fungsi embed_from_path() dari utils_facenet.py. Jika salah satu gambar gagal mendeteksi wajah, program segera menghentikan proses dan menampilkan peringatan.
Ketika kedua embedding berhasil dihasilkan, script menghitung nilai kesamaan menggunakan cosine similarity, metode yang sangat umum dalam sistem FaceNet untuk mengukur kedekatan dua embedding wajah. Nilai similarity dicetak sehingga pengguna dapat melihat seberapa mirip kedua wajah tersebut dalam bentuk angka. Setelah itu, nilai similarity dibandingkan dengan threshold 0.85, yang menjadi batas apakah dua wajah dianggap identik atau tidak. Jika similarity lebih tinggi atau sama dengan threshold, program menampilkan “YA” sebagai indikator kecocokan, jika tidak maka “TIDAK”.
Script ini sangat cocok untuk pengujian cepat, demonstrasi hasil embedding, dan eksperimen threshold karena sederhana, langsung, dan tidak memerlukan input tambahan. Ini juga berguna sebagai langkah debugging awal sebelum mengintegrasikan sistem verifikasi ke aplikasi yang lebih besar.